In [631]:

import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

In [633]:
# # Load environment variables from .env file
load_dotenv()

True

In [634]:
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))


In [635]:
raw_text= """ 

Dear Arko,
I hope you're doing well! I recently visited Shimla and had an amazing time, so I wanted to
share a bit about my trip with you.
The journey started with the toy train ride from Kalka, which was an incredible experience
with its scenic views and winding tracks. Shimla itself is beautiful, with its colonial
architecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling
along Mall Road, exploring shops, and indulging in local cuisine.
A visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along
the way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried
horse riding.
The locals were warm and welcoming, adding to the charm of the trip. Shimla was the
perfect escape from the city's hustle and bustle.
I wish you could have been there to share the experience. Let's plan a trip together soon!
Take care and stay in touch!
Warm regards,
Swastik Dhar


"""

In [636]:
question = "How was the trip for Swastik Dhar? "

In [637]:
# Split the text into chunks using a character-based splitter
    # The chunks should be small enough to avoid exceeding token limits
text_splitter = CharacterTextSplitter(
        separator="\n",  # Split the text at newline characters
        chunk_size=10000,  # Maximum size of each chunk
        chunk_overlap=800,  # Overlap between consecutive chunks
        length_function=len,  # Function to calculate the length of each chunk
    )
texts = text_splitter.split_text(raw_text)  # Split the raw text into chunks
texts

["Dear Arko,\nI hope you're doing well! I recently visited Shimla and had an amazing time, so I wanted to\nshare a bit about my trip with you.\nThe journey started with the toy train ride from Kalka, which was an incredible experience\nwith its scenic views and winding tracks. Shimla itself is beautiful, with its colonial\narchitecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling\nalong Mall Road, exploring shops, and indulging in local cuisine.\nA visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along\nthe way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried\nhorse riding.\nThe locals were warm and welcoming, adding to the charm of the trip. Shimla was the\nperfect escape from the city's hustle and bustle.\nI wish you could have been there to share the experience. Let's plan a trip together soon!\nTake care and stay in touch!\nWarm regards,\nSwastik Dhar"]

In [638]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

embeddings


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000014700198910>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [639]:
document_search = FAISS.from_texts(texts, embeddings)
document_search


In [640]:
docs = document_search.similarity_search(question)
docs

[Document(page_content="Dear Arko,\nI hope you're doing well! I recently visited Shimla and had an amazing time, so I wanted to\nshare a bit about my trip with you.\nThe journey started with the toy train ride from Kalka, which was an incredible experience\nwith its scenic views and winding tracks. Shimla itself is beautiful, with its colonial\narchitecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling\nalong Mall Road, exploring shops, and indulging in local cuisine.\nA visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along\nthe way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried\nhorse riding.\nThe locals were warm and welcoming, adding to the charm of the trip. Shimla was the\nperfect escape from the city's hustle and bustle.\nI wish you could have been there to share the experience. Let's plan a trip together soon!\nTake care and stay in touch!\nWarm regards,\nSwastik Dhar")]

In [641]:
prompt = "Hey Can you answer me the following question? My question is: "+question + "also I got the following text for you help. Following text: "+str(docs[0])
prompt

'Hey Can you answer me the following question? My question is: How was the trip for Swastik Dhar? also I got the following text for you help. Following text: page_content="Dear Arko,\\nI hope you\'re doing well! I recently visited Shimla and had an amazing time, so I wanted to\\nshare a bit about my trip with you.\\nThe journey started with the toy train ride from Kalka, which was an incredible experience\\nwith its scenic views and winding tracks. Shimla itself is beautiful, with its colonial\\narchitecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling\\nalong Mall Road, exploring shops, and indulging in local cuisine.\\nA visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along\\nthe way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried\\nhorse riding.\\nThe locals were warm and welcoming, adding to the charm of the trip. Shimla was the\\nperfect escape from the city\'s hustle and 

In [642]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-pro',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
    ),
    history=[]
)

In [643]:
response = chat.send_message(prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The given text does not provide any information about Swastik Dhar's trip."
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 9,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 8,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 7,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 10,
              "probability": 1,
              "blocked": false
            }
          ],
          "token_count": 0,
          "groun

In [644]:
import textwrap


response = chat.send_message(prompt+"Are you sure?", stream=True)
response
x=""
for chunk in response:
  x+=chunk.text

x = x.replace('*', '')
x = x.replace('**','')
x
  
  

'I am sure that the given text does not provide any information about how the trip was for Swastik Dhar. The text only describes the places that Swastik Dhar visited and the activities they engaged in during their trip to Shimla. It does not provide any subjective评价 or感想s about the trip experience itself.'

In [645]:
response = chat.send_message(prompt+"No it was the Swastik Dhar who is sharing his travelling expreience.", stream=True)
for chunk in response:
  print(chunk.text)
  

The provided text is written by Swastik Dhar, who is sharing their travel
 experience in Shimla with the recipient Arko. However, the text does not explicitly state how the trip was for Swastik Dhar. It only describes the
 places they visited and the activities they engaged in during their trip.

Without additional information, we cannot determine how Swastik Dhar felt about their trip.


In [646]:
chat.history

[parts {
   text: "Hey Can you answer me the following question? My question is: How was the trip for Swastik Dhar? also I got the following text for you help. Following text: page_content=\"Dear Arko,\\nI hope you\'re doing well! I recently visited Shimla and had an amazing time, so I wanted to\\nshare a bit about my trip with you.\\nThe journey started with the toy train ride from Kalka, which was an incredible experience\\nwith its scenic views and winding tracks. Shimla itself is beautiful, with its colonial\\narchitecture and vibrant streets. The Ridge offered breathtaking views, and I loved strolling\\nalong Mall Road, exploring shops, and indulging in local cuisine.\\nA visit to Jakhoo Temple was a highlight, with stunning city views and playful monkeys along\\nthe way. I also took a short trip to Kufri, where I enjoyed the serene beauty and even tried\\nhorse riding.\\nThe locals were warm and welcoming, adding to the charm of the trip. Shimla was the\\nperfect escape from the 

In [647]:
chat.history.clear()
chat.history

[]

In [648]:
 # Ensure the OpenAI API key is set
#     api_key = os.getenv("OPENAI_API_KEY")
#     if not api_key:
#         raise ValueError("OPENAI_API_KEY environment variable not set")

#     # Initialize the OpenAI object with the API key
#     openai = OpenAI(api_key=api_key)

#     # Split the text into chunks using a character-based splitter
#     # The chunks should be small enough to avoid exceeding token limits
#     text_splitter = CharacterTextSplitter(
#         separator="\n",  # Split the text at newline characters
#         chunk_size=800,  # Maximum size of each chunk
#         chunk_overlap=200,  # Overlap between consecutive chunks
#         length_function=len,  # Function to calculate the length of each chunk
#     )
#     texts = text_splitter.split_text(raw_text)  # Split the raw text into chunks

#     # Download embeddings from OpenAI
#     embeddings = OpenAIEmbeddings(api_key=api_key)

#     # Create a FAISS index from the text chunks and their embeddings
#     document_search = FAISS.from_texts(texts, embeddings)

#     # Load the question-answering chain with the OpenAI model
#     chain = load_qa_chain(openai, chain_type="stuff")

#     # Perform a similarity search on the documents to find the most relevant chunks
#     docs = document_search.similarity_search(question)
    
#     # Use the question-answering chain to generate an answer based on the relevant documents
#     result = chain.invoke({"input_documents": docs, "question": question})
    
#     # Extract the answer text from the result
#     answer = result["output_text"]
    
#     return answer